In [ ]:
import spacy
import pandas as pd
import numpy as np
from spacy import displacy
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
# set up the entity ruler before the ner in pipeline
ruler = nlp.add_pipe("entity_ruler", before="ner")

In [ ]:
# instructions:
# review if text uses period after Mr Mrs etc and replace as needed
# look at top line for generic names to see if some need to be commented out
ruler_patterns = {
    'oswald' : [ #lee harvey oswald, assassinated jfk
        {"label": "PERSON", "pattern": "Lee Harvey Oswald", "id": "Lee Harvey Oswald"},
        {"label": "PERSON", "pattern": "Lee Oswald", "id": "Lee Harvey Oswald"},
        {"label": "PERSON", "pattern": "Lee", "id": "Lee Harvey Oswald"},
        {"label": "PERSON", "pattern": "Oswald", "id": "Lee Harvey Oswald"},
    ],
    'jfk' : [ #john f. kennedy, us president, incl kennedy
        {"label": "PERSON", "pattern": [{"LOWER": "jfk"}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": [{"LOWER": "kennedy"}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": [{"LOWER": "john"}, {"LOWER": "kennedy"}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": [{"LOWER": "president"}, {"LOWER": "kennedy"}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": [{"LOWER": "john"}, {"LOWER": "f."}, {"LOWER": "kennedy"}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": [{"LOWER": "j.f.k."}], "id": "John F. Kennedy"},
        {"label": "PERSON", "pattern": "John Fitzgerald", "id": "John F. Kennedy"},
    ],
    'onassis' : [ #jacqueline kennedy onassis, us first lady, incl jackie
        {"label": "PERSON", "pattern": "Mrs Kennedy", "id": "Jacqueline Kennedy Onassis"},
        {"label": "PERSON", "pattern": "Jacqueline", "id": "Jacqueline Kennedy Onassis"},
        {"label": "PERSON", "pattern": "Jackie Kennedy", "id": "Jacqueline Kennedy Onassis"},
        {"label": "PERSON", "pattern": "Jackie", "id": "Jacqueline Kennedy Onassis"},
    ],
    'ferrie' : [ #david ferrie, american pilot, includes Dave
        {"label": "PERSON", "pattern": "Ferrie", "id": "David Ferrie"},
        {"label": "PERSON", "pattern": "David Ferrie", "id": "David Ferrie"},
        {"label": "PERSON", "pattern": "Dave Ferrie", "id": "David Ferrie"},
        {"label": "PERSON", "pattern": "Dave", "id": "David Ferrie"},
    ],
    #'shaw' : [ #clay shaw, american businessman, incl shaw, not mentioned in this book
       # {"label": "PERSON", "pattern": "Shaw", "id": "Clay Shaw"},
       # {"label": "PERSON", "pattern": "Clay Shaw", "id": "Clay Shaw"},
 #   ],
    'fbi' : [
        {"label": "ORG", "pattern": "FBI", "id": "Federal Bureau of Investigation"},
        {"label": "ORG", "pattern": "F.B.I.", "id": "Federal Bureau of Investigation"},
        {"label": "ORG", "pattern": "The Bureau", "id": "Federal Bureau of Investigation"},
        {"label": "ORG", "pattern": "the Bureau", "id": "Federal Bureau of Investigation"},
        {"label": "ORG", "pattern": "Bureau", "id": "Federal Bureau of Investigation"},
    ],
    'wcomm' : [
        {"label": "ORG", "pattern": "Warren Commission", "id": "Warren Commission"},
        {"label": "ORG", "pattern": "The Warren Commission", "id": "Warren Commission"},
        {"label": "ORG", "pattern": "the Warren Commission", "id": "Warren Commission"},
        {"label": "ORG", "pattern": "Commission", "id": "Warren Commission"},
    ],
    'cia' : [
        {"label": "ORG", "pattern": "CIA", "id": "Central Intelligence Agency"},
        {"label": "ORG", "pattern": "C.I.A.", "id": "Central Intelligence Agency"},
        {"label": "ORG", "pattern": "The Agency", "id": "Central Intelligence Agency"},
        {"label": "ORG", "pattern": "the Agency", "id": "Central Intelligence Agency"},
        {"label": "ORG", "pattern": "Agency", "id": "Central Intelligence Agency"},
        {"label": "ORG", "pattern": "the Central Intelligence", "id": "Central Intelligence Agency"},
    ],
    'gervais' : [ # pershing gervais, worked for garrison
        {"label": "PERSON", "pattern": "Pershing Gervais", "id": "Pershing Gervais"},
        {"label": "PERSON", "pattern": "Gervais", "id": "Pershing Gervais"},
    ],
    'russo' : [ #perry russo, american insurance salesman
        {"label": "PERSON", "pattern": "Perry Russo", "id": "Perry Russo"},
        {"label": "PERSON", "pattern": "Parry Raymond Russo", "id": "Perry Russo"},
        {"label": "PERSON", "pattern": "Russo", "id": "Perry Russo"},
    ],
    'thornley' : [ #kerry wendell thornley, american author
        {"label": "PERSON", "pattern": "Kerry Thornley", "id": "Kerry Wendell Thornley"},
        {"label": "PERSON", "pattern": "Kerry Wendell Thornley", "id": "Kerry Wendell Thornley"},
        {"label": "PERSON", "pattern": "Thornley", "id": "Kerry Wendell Thornley"},
    ],
    'whalen' : [ 
        {"label": "PERSON", "pattern": "Edward Whalen", "id": "Edward James Whalen"},
        {"label": "PERSON", "pattern": "Whalen", "id": "Edward James Whalen"},
    ],
    'banister' : [ #william guy banister, fbi agent
        {"label": "PERSON", "pattern": "Guy Banister", "id": "William Guy Banister"},
        {"label": "PERSON", "pattern": "William Guy Banister", "id": "William Guy Banister"},
        {"label": "PERSON", "pattern": "Banister", "id": "William Guy Banister"},
    ],
    'tippit' : [ #j.d. tippit, dallas pd
        {"label": "PERSON", "pattern": "J.D. Tippit", "id": "J.D. Tippit"},
        {"label": "PERSON", "pattern": "Tippit", "id": "J.D. Tippit"},
        {"label": "PERSON", "pattern": "J. D. Tippit", "id": "J.D. Tippit"},
        {"label": "PERSON", "pattern": "JD Tippit", "id": "J.D. Tippit"},
    ],
    'sciambra' : [ # andrew sciambra, assistant da, incl andrew
        {"label": "PERSON", "pattern": "Andrew Sciambra", "id": "Andrew Sciambra"},
        {"label": "PERSON", "pattern": "Sciambra", "id": "Andrew Sciambra"},
        {"label": "PERSON", "pattern": "Andrew", "id": "Andrew Sciambra"},
    ],
    'dean' : [ #dean andrews, new orleans attorney, not incl dean for lane_rush , incl andrews
        {"label": "PERSON", "pattern": "Dean Andrews", "id": "Dean Adams Andrews, Jr."},
        #{"label": "PERSON", "pattern": "Dean", "id": "Dean Adams Andrews, Jr."},
        {"label": "PERSON", "pattern": "Andrews", "id": "Dean Adams Andrews, Jr."},
        {"label": "PERSON", "pattern": "Dean Adams Andrews, Jr.", "id": "Dean Adams Andrews, Jr."},
    ],
    'bertrand' : [ #clay bertrand, alias used by unknown
        {"label": "PERSON", "pattern": "Clay Bertrand", "id": "Clay Bertrand"},
        {"label": "PERSON", "pattern": "Bertrand", "id": "Clay Bertrand"},
    ],
    'jgarr' : [ # jim garrison, louisianna attorney, incl jim
        {"label": "PERSON", "pattern": "Garrison", "id": "Jim Garrison"},
        {"label": "PERSON", "pattern": "James Carothers Garrison", "id": "Jim Garrison"},
        {"label": "PERSON", "pattern": "Earling Carothers Garrison", "id": "Jim Garrison"},
        {"label": "PERSON", "pattern": "James Carothers Garrison", "id": "Jim Garrison"},
        {"label": "PERSON", "pattern": "James Garrison", "id": "Jim Garrison"},
        #{"label": "PERSON", "pattern": "Jim", "id": "Jim Garrison"},
    ],
    'oser' : [
        {"label": "PERSON", "pattern": "Alvin Oser", "id": "Alvin Oser"},
        {"label": "PERSON", "pattern": "Oser", "id": "Alvin Oser"},
        {"label": "PERSON", "pattern": "Al Oser", "id": "Alvin Oser"},
        {"label": "PERSON", "pattern": "OSER", "id": "Alvin Oser"},
    ],
    'bundy' : [
        {"label": "PERSON", "pattern": "Vernon Bundy", "id": "Vernon Bundy"},
        {"label": "PERSON", "pattern": "Bundy", "id": "Vernon Bundy"},
    ],
    'klein' : [
        {"label": "PERSON", "pattern": "Frank Klein", "id": "Frank Klein"},
        {"label": "PERSON", "pattern": "Klein", "id": "Frank Klein"},
        {"label": "PERSON", "pattern": "Frank", "id": "Frank Klein"},
    ],
    'ruby' : [
        {"label": "PERSON", "pattern": "Jack Ruby", "id": "Jack Ruby"},
        {"label": "PERSON", "pattern": "Ruby", "id": "Jack Ruby"},
    ],
#    'martin' : [ # ambiguous in lane rush, a few characters equally addressed with martin surname
#        {"label": "PERSON", "pattern": "Jack Martin", "id": "Jack Martin"},
    
#        {"label": "PERSON", "pattern": "Martin", "id": "Jack Martin"},
#        {"label": "PERSON", "pattern": "Jack", "id": "Jack Martin"},
#    ],
    'mlk' : [
        {"label": "PERSON", "pattern": "Martin Luther King, Jr.", "id": "Martin Luther King, Jr."},
        {"label": "PERSON", "pattern": "Martin Luther King", "id": "Martin Luther King, Jr."},
    ],
    'ivon' : [
        {"label": "PERSON", "pattern": "Louis Ivon", "id": "Louis Ivon"},
        {"label": "PERSON", "pattern": "Lou Ivon", "id": "Louis Ivon"},
        {"label": "PERSON", "pattern": "Ivon", "id": "Louis Ivon"},
        {"label": "PERSON", "pattern": "Lou", "id": "Louis Ivon"},
    ],
    'sserv' : [
        {"label": "ORG", "pattern": "Secret Service", "id": "United States Secret Service"},
        {"label": "ORG", "pattern": "the Secret Service", "id": "United States Secret Service"},
        {"label": "ORG", "pattern": "The Secret Service", "id": "United States Secret Service"},
    ],
    'pizzo' : [
        {"label": "PERSON", "pattern": "Frank Pizzo", "id": "Frank Pizzo"},
    ],
    'finck' : [
        {"label": "PERSON", "pattern": "Lieutenant Colonel Pierre Finck", "id": "Pierre Finck"},
        {"label": "PERSON", "pattern": "Lieutenant Colonel Finck", "id": "Pierre Finck"},
        {"label": "PERSON", "pattern": "Dr Finck", "id": "Pierre Finck"},
        {"label": "PERSON", "pattern": "Finck", "id": "Pierre Finck"},
        {"label": "PERSON", "pattern": "FINCK", "id": "Pierre Finck"},
    ],
   'craig' : [ # Roger Craig, officer dallas pd, at jfk shooting
        {"label": "PERSON", "pattern": "Deputy Sherriff Roger Craig", "id": "Roger Craig"},
        {"label": "PERSON", "pattern": "Roger Craig", "id": "Roger Craig"},
        {"label": "PERSON", "pattern": "Craig", "id": "Roger Craig"},
    ],
    'boxley' : [
        {"label": "PERSON", "pattern": "Bill Boxley", "id": "William Wood"},
        {"label": "PERSON", "pattern": "Boxley", "id": "William Wood"},
        {"label": "PERSON", "pattern": "Wood", "id": "William Wood"},
    ],
    'nagell' : [
        {"label": "PERSON", "pattern": "Richard Case Nagell", "id": "Richard Case Nagell"},
        {"label": "PERSON", "pattern": "Nagell", "id": "Richard Case Nagell"},
    ],
    'carson' : [
        {"label": "PERSON", "pattern": "Johnny Carson", "id": "Johnny Carson"},
        {"label": "PERSON", "pattern": "Carson", "id": "Johnny Carson"},
        {"label": "PERSON", "pattern": "John William Carson", "id": "Johnny Carson"},
    ],
    'castro' : [
        {"label": "PERSON", "pattern": "Fidel Castro", "id": "Fidel Castro"},
        {"label": "PERSON", "pattern": "Castro", "id": "Fidel Castro"},
    ],
    'spiesel' : [
        {"label": "PERSON", "pattern": "Charles Spiesel", "id": "Charles Spiesel"},
        {"label": "PERSON", "pattern": "Spiesel", "id": "Charles Spiesel"},
    ],
    'kimble' : [
        {"label": "PERSON", "pattern": "Jules Ricco Kimble", "id": "Jules Ricco Kimble"},
        {"label": "PERSON", "pattern": "Kimble", "id": "Jules Ricco Kimble"},
    ],
    'lbj' : [ #note lbj includes `Johnson` so keep that in mind
        {"label": "PERSON", "pattern": "Lyndon Johnson", "id": "Lyndon B. Johnson"},
        {"label": "PERSON", "pattern": "President Lyndon Johnson", "id": "Lyndon B. Johnson"},
        {"label": "PERSON", "pattern": "Johnson", "id": "Lyndon B. Johnson"},  
    ],
    'guyj' : [
        {"label": "PERSON", "pattern": "Guy Johnson", "id": "Guy Johnson"},   
    ],
    'jimmyj' : [ # not seemingly significant person but generic name `Jimmy Johnson`
        {"label": "PERSON", "pattern": "Jimmy Johnson", "id": "Jimmy Johnson"},    
    ],
    'rosselli' : [
        {"label": "PERSON", "pattern": "John Rosselli", "id": "John Roselli"},    
        {"label": "PERSON", "pattern": "John Roselli", "id": "John Roselli"},  
        {"label": "PERSON", "pattern": "Roselli", "id": "John Roselli"},  
        {"label": "PERSON", "pattern": "Rosselli", "id": "John Roselli"},
    ],
    'alcock' : [
        {"label": "PERSON", "pattern": "James Alcock", "id": "James Alcock"}, 
        {"label": "PERSON", "pattern": "Jim Alcock", "id": "James Alcock"},    
        {"label": "PERSON", "pattern": "Alcock", "id": "James Alcock"},    
    ],
    'drnichols' : [ # generic name, gave deposition Garrison Trails
        {"label": "PERSON", "pattern": "Dr John Nichols", "id": "John Nichols"}, 
        {"label": "PERSON", "pattern": "Dr Nichols", "id": "John Nichols"}, 
    ],
    'marina' : [
        {"label": "PERSON", "pattern": "Marina", "id": "Marina Oswald Porter"},
        {"label": "PERSON", "pattern": "Marina Prusakova", "id": "Marina Oswald Porter"},
        {"label": "PERSON", "pattern": "Marina Porter", "id": "Marina Oswald Porter"},
        {"label": "PERSON", "pattern": "Marina Oswald", "id": "Marina Oswald Porter"},
    ],
    'ruth' : [ # not including Paine because husband Michael Paine
        {"label": "PERSON", "pattern": "Ruth Paine", "id": "Ruth Paine"}, 
        {"label": "PERSON", "pattern": "Ruth", "id": "Ruth Paine"},
        {"label": "PERSON", "pattern": "Ruth Hyde Paine", "id": "Ruth Paine"},
        {"label": "PERSON", "pattern": "Ruth Avery Hyde", "id": "Ruth Paine"},
    ],
    'justdept' : [
        {"label": "ORG", "pattern": "the Justice Department", "id": "United States Department of Justice"},
        {"label": "ORG", "pattern": "The Justice Department", "id": "United States Department of Justice"},
        {"label": "ORG", "pattern": "Justice Department", "id": "United States Department of Justice"},
        {"label": "ORG", "pattern": "the United States Justice Department", "id": "United States Department of Justice"},
    ],
    'hsca' : [
        {"label": "ORG", "pattern": "the House Select Committee on Assassinations", "id": "House Select Committee on Assassinations"},
    ],
    'dulles' : [
        {"label": "PERSON", "pattern": "Allen Dulles", "id": "Allen W. Dulles"}, 
        {"label": "PERSON", "pattern": "Dulles", "id": "Allen W. Dulles"}, 
        {"label": "PERSON", "pattern": "Allen W Dulles", "id": "Allen W. Dulles"}, 
    ],
    'jamercer' : [
        {"label": "PERSON", "pattern": "Julia Ann Mercer", "id": "Julia Ann Mercer"}, 
        {"label": "PERSON", "pattern": "Mercer", "id": "Julia Ann Mercer"}, 
    ],
    'fritz' : [
        {"label": "PERSON", "pattern": "Captain Will Fritz", "id": "John William Fritz"}, 
        {"label": "PERSON", "pattern": "Will Fritz", "id": "John William Fritz"}, 
        {"label": "PERSON", "pattern": "Captain Fritz", "id": "John William Fritz"}, 
        {"label": "PERSON", "pattern": "Fritz", "id": "John William Fritz"},
    ],
    'mohrenschildt' : [
        {"label": "PERSON", "pattern": "George de Mohrenschildt", "id": "George de Mohrenschildt"},
        {"label": "PERSON", "pattern": "de Mohrenschildt", "id": "George de Mohrenschildt"},
        {"label": "PERSON", "pattern": "De Mohrenschildt", "id": "George de Mohrenschildt"},
        {"label": "PERSON", "pattern": "Baron George de Mohrenschildt", "id": "George de Mohrenschildt"},  
    ],
    'carr' : [
        {"label": "PERSON", "pattern": "Richard Randolph Carr", "id": "Richard Randolph Carr"},
        {"label": "PERSON", "pattern": "Carr", "id": "Richard Randolph Carr"},
    ],
    'phelan' : [
        {"label": "PERSON", "pattern": "James Phelan", "id": "James Phelan"},
        {"label": "PERSON", "pattern": "Phelan", "id": "James Phelan"},
    ],
    'hosty' : [ 
        {"label": "PERSON", "pattern": "James Hosty", "id": "James P. Hosty"},  
        {"label": "PERSON", "pattern": "Hosty", "id": "James P. Hosty"},  
        {"label": "PERSON", "pattern": "Agent James Hosty", "id": "James P. Hosty"},  
        {"label": "PERSON", "pattern": "Agent Hosty", "id": "James P. Hosty"}, 
    ],
    'dymond' : [
        {"label": "PERSON", "pattern": "F. Irvin Dymond", "id": "F. Irvin Dymond"},
        {"label": "PERSON", "pattern": "Irvin Dymond", "id": "F. Irvin Dymond"},
        {"label": "PERSON", "pattern": "Dymond", "id": "F. Irvin Dymond"},
    ],
    'lvmeyers' : [ #knew Ruby
        {"label": "PERSON", "pattern": "Lawrence V. Meyers", "id": "Lawrence V. Meyers"},
        {"label": "PERSON", "pattern": "Meyers", "id": "Lawrence V. Meyers"},
        {"label": "PERSON", "pattern": "Lawrence Meyers", "id": "Lawrence V. Meyers"},
    ],
    'aase' : [ #Jean Aase mentioned in Commission report, many aka's in Garrison
        {"label": "PERSON", "pattern": "Miss A. Asie", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Miss Asie", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Jean Aase", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Aase", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Miss Aase", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Jean West", "id": "Jean Aase"},
        {"label": "PERSON", "pattern": "Miss West", "id": "Jean Aase"},
    ],
    'johnmiller' : [ #garrison claims some oil man from Denver, not incl miller
        {"label": "PERSON", "pattern": "John Miller", "id": "John Miller"},
        {"label": "PERSON", "pattern": "Mr Miller", "id": "John Miller"},      
    ],
    'jehoover' : [
        {"label": "PERSON", "pattern": "J. Edgar Hoover", "id": "J. Edgar Hoover"},
        {"label": "PERSON", "pattern": "Edgar Hoover", "id": "J. Edgar Hoover"},
        {"label": "PERSON", "pattern": "Hoover", "id": "J. Edgar Hoover"},
    ],
    'gnovel' : [ #Gordon Novel, private investigator
        {"label": "PERSON", "pattern": "Gordon Novel", "id": "Gordon Novel"},
        {"label": "PERSON", "pattern": "Novel", "id": "Gordon Novel"},
    ],
    'mrsfarrington' : [
        {"label": "PERSON", "pattern": "Fenella Farrington", "id": "Fenella Farrington"},
        {"label": "PERSON", "pattern": "Mrs Fenella Farrington", "id": "Fenella Farrington"},
        {"label": "PERSON", "pattern": "Mrs Farrington", "id": "Fenella Farrington"},
    ],
    'habighorst' : [
        {"label": "PERSON", "pattern": "Aloysius Habighorst", "id": "Aloysius Habighorst"},
        {"label": "PERSON", "pattern": "Habighorst", "id": "Aloysius Habighorst"},
        {"label": "PERSON", "pattern": "Officer Habighorst", "id": "Aloysius Habighorst"},
    ],
    'sheridan' : [
        {"label": "PERSON", "pattern": "Walter Sheridan", "id": "Walter Sheridan"},
        {"label": "PERSON", "pattern": "Sheridan", "id": "Walter Sheridan"},
    ],
    'salandria' : [
        {"label": "PERSON", "pattern": "Vincent Salandria", "id": "Vincent Salandria"},
        {"label": "PERSON", "pattern": "Salandria", "id": "Vincent Salandria"},
    ],
    'willwalter' : [
        {"label": "PERSON", "pattern": "William S. Walter", "id": "William S. Walter"},
        {"label": "PERSON", "pattern": "William Walter", "id": "William S. Walter"},
        {"label": "PERSON", "pattern": "Walter", "id": "William S. Walter"},
    ],
    'haggerty' : [
        {"label": "PERSON", "pattern": "Edward Aloysius Haggerty", "id": "Edward Aloysius Haggerty"}, 
        {"label": "PERSON", "pattern": "Judge Haggerty", "id": "Edward Aloysius Haggerty"}, 
        {"label": "PERSON", "pattern": "Haggerty", "id": "Edward Aloysius Haggerty"}, 
    ],
    'humes' : [
        {"label": "PERSON", "pattern": "James J. Humes", "id": "James J. Humes"}, 
        {"label": "PERSON", "pattern": "Humes", "id": "James J. Humes"}, 
        {"label": "PERSON", "pattern": "Commander Humes", "id": "James J. Humes"}, 
        {"label": "PERSON", "pattern": "Commander James Humes", "id": "James J. Humes"}, 
        {"label": "PERSON", "pattern": "Dr Humes", "id": "James J. Humes"}, 
    ],
    'rosemaryjames' : [
        {"label": "PERSON", "pattern": "Rosemary James", "id": "Rosemary James"},
    ],
    'justicewarren' : [
        {"label": "PERSON", "pattern": "Chief Justice Earl Warren", "id": "Earl Warren"},
        {"label": "PERSON", "pattern": "Earl Warren", "id": "Earl Warren"},
        {"label": "PERSON", "pattern": "Chief Justice Warren", "id": "Earl Warren"},
        {"label": "PERSON", "pattern": "Justice Warren", "id": "Earl Warren"},
    ],
    'mlane' : [
        {"label": "PERSON", "pattern": "Mark Lane", "id": "Mark Lane"},
        {"label": "PERSON", "pattern": "Lane", "id": "Mark Lane"},
    ],
    'euins' : [
        {"label": "PERSON", "pattern": "Amos Euins", "id": "Amos Lee Euins"},
        {"label": "PERSON", "pattern": "Euins", "id": "Amos Lee Euins"},
    ],
    'ecabell' : [ # Earle Cabell - cannot use just Cabell due to brother Charles
        {"label": "PERSON", "pattern": "Earle Cabell", "id": "Earle Cabell"},
        {"label": "PERSON", "pattern": "Mayor Cabell", "id": "Earle Cabell"},
    ],
    'ccabell' : [ # Charles Cabell - cannot use just Cabell due to brother Earle
        {"label": "PERSON", "pattern": "Charles Cabell", "id": "Charles Cabell"},
        {"label": "PERSON", "pattern": "General Cabell", "id": "Earle Cabell"},
    ],
    'sprague' : [
        {"label": "PERSON", "pattern": "Richard E. Sprague", "id": "Richard E. Sprague"},
        {"label": "PERSON", "pattern": "Sprague", "id": "Richard E. Sprague"},
    ],
    'christenberry' : [
        {"label": "PERSON", "pattern": "Herbert Christenberry", "id": "Herbert William Christenberry"},
        {"label": "PERSON", "pattern": "Judge Christenberry", "id": "Herbert William Christenberry"},
    ],
    'lumumba' : [
        {"label": "PERSON", "pattern": "Patrice Lumumba", "id": "Patrice Lumumba"},
        {"label": "PERSON", "pattern": "Lumumba", "id": "Patrice Lumumba"},
    ],
    'helms' : [
        {"label": "PERSON", "pattern": "Richard McGarrah Helms", "id": "Richard McGarrah Helms"},
        {"label": "PERSON", "pattern": "Richard Helms", "id": "Richard McGarrah Helms"},
        {"label": "PERSON", "pattern": "Helms", "id": "Richard McGarrah Helms"},
    ],
    'mpaine' : [ # husband of Ruth Paine
        {"label": "PERSON", "pattern": "Michael Paine", "id": "Michael Paine"},
    ],
    'ewilson' : [ # eugene wilson, car salesman
        {"label": "PERSON", "pattern": "Eugene Wilson", "id": "Eugene Wilson"},
        {"label": "PERSON", "pattern": "Eugene M. Wilson", "id": "Eugene Wilson"},
        #{"label": "PERSON", "pattern": "Wilson", "id": "Eugene Wilson"},
    ],
    'fritz' : [
        {"label": "PERSON", "pattern": "Will Fritz", "id": "John Will Fritz"},
        {"label": "PERSON", "pattern": "Captain Fritz", "id": "John Will Fritz"},
        {"label": "PERSON", "pattern": "Captain Will Fritz", "id": "John Will Fritz"},
        {"label": "PERSON", "pattern": "Fritz", "id": "John Will Fritz"},
    ],
    'reily' : [ #william reily, does not include reily bc its a coffee company
        {"label": "PERSON", "pattern": "William Reily", "id": "William Reily"},
    ],
    'ndelgado' : [ # nelson delgado, served with oswald, includes delgado
        {"label": "PERSON", "pattern": "Nelson Delgado", "id": "Nelson Delgado"},
        {"label": "PERSON", "pattern": "Delgado", "id": "Nelson Delgado"},
    ],
    'mclark' : [ # max clark, former air force colonel, not incl clark
        {"label": "PERSON", "pattern": "Max Clark", "id": "Max Clark"},
        {"label": "PERSON", "pattern": "Colonel Max Clark", "id": "Max Clark"},
        {"label": "PERSON", "pattern": "Colonel Clark", "id": "Max Clark"},
    ],
    'rclark' : [ # ramsey clark, attorney general, not incl clark
        {"label": "PERSON", "pattern": "Ramsey Clark", "id": "William Ramsey Clark"},
    ],
    'andrewjackson' : [ # not incl jackson
        {"label": "PERSON", "pattern": "General Andrew Jackson", "id": "Andrew Jackson"},
    ],
    'daltonwi' : [ #d'alton williams, not incl williams
        {"label": "PERSON", "pattern": "D'Alton Williams", "id": "D'Alton Williams"},
        {"label": "PERSON", "pattern": "D'Alton", "id": "D'Alton Williams"},
    ],
    'aynesworth' : [
        {"label": "PERSON", "pattern": "Hugh Aynesworth", "id": "Hugh Aynesworth"},
        {"label": "PERSON", "pattern": "Aynesworth", "id": "Hugh Aynesworth"},
    ],
    'eisenhower' : [ #dwight d eisenhower, us president, incl ike
        {"label": "PERSON", "pattern": "President Eisenhower", "id": "Dwight D. Eisenhower"},
        {"label": "PERSON", "pattern": "Eisenhower", "id": "Dwight D. Eisenhower"},
        {"label": "PERSON", "pattern": [{"TEXT": "Ike"}], "id": "Dwight D. Eisenhower"},
    ],
    'markham' : [ # helen markham, witness to jd tippitt shooting
        {"label": "PERSON", "pattern": "Helen Markham", "id": "Helen Markham"},
        {"label": "PERSON", "pattern": "Markham", "id": "Helen Markham"},
        {"label": "PERSON", "pattern": "Mrs Markham", "id": "Helen Markham"},
    ],
    'fwright' : [ # frank wright, a mrs. wright is also mentioned, witness to jd tippitt shooting
        {"label": "PERSON", "pattern": "Frank Wright", "id": "Frank Wright"},
        {"label": "PERSON", "pattern": "Wright", "id": "Frank Wright"},
        {"label": "PERSON", "pattern": "Mr Wright", "id": "Frank Wright"},
    ],
    'mrswright' : [ #wife to frank wright, first name not mentioned
        {"label": "PERSON", "pattern": "Mrs Wright", "id": "Mrs. Wright"},
    ],
    'hamblen' : [ # c.a. hamblen, western telegraph co
        {"label": "PERSON", "pattern": "C.A. Hamblen", "id": "C. A. Hamblen"},
        {"label": "PERSON", "pattern": "Hamblen", "id": "C. A. Hamblen"},
    ],
    'darrow' : [ # clarence darrow, lawyer - not incl darrow bc name of garrison child
        {"label": "PERSON", "pattern": "Clarence Darrow", "id": "Clarence Darrow"},
    ],
    'fnagy' : [ # ferenc nagy, hungarian politician
        {"label": "PERSON", "pattern": "Ferenc Nagy", "id": "Ferenc Nagy"},
        {"label": "PERSON", "pattern": "Nagy", "id": "Ferenc Nagy"},
    ],
    'bloomfield' : [ #lm bloomfield, canadian lawyer businessman soldier
        {"label": "PERSON", "pattern": "L.M. Bloomfield", "id": "Louis Mortimer Bloomfield"},
        {"label": "PERSON", "pattern": "Major L.M. Bloomfield", "id": "Louis Mortimer Bloomfield"},
        {"label": "PERSON", "pattern": "Bloomfield", "id": "Louis Mortimer Bloomfield"},
    ],
    'spencer' : [ # john spencer, knew thornley, shaw, incl spencer
        {"label": "PERSON", "pattern": "John Spencer", "id": "John Spencer"},
        {"label": "PERSON", "pattern": "Spencer", "id": "John Spencer"},
    ],
    'jconnally' : [ #texas gov, incl connally
        {"label": "PERSON", "pattern": "John Connally", "id": "John Connally"},
        {"label": "PERSON", "pattern": "Governor Connally", "id": "John Connally"},
        {"label": "PERSON", "pattern": "Governor John Connally", "id": "John Connally"},
        {"label": "PERSON", "pattern": "Connally", "id": "John Connally"},
    ],
    'jpowell' : [ # john powell, jfk shooting witness, incl powell
        {"label": "PERSON", "pattern": "John Powell", "id": "John Powell"},
        {"label": "PERSON", "pattern": "Powell", "id": "John Powell"},
    ],
    'wgill' : [ # wray gill, louisianna lawyer
        {"label": "PERSON", "pattern": "Wray Gill", "id": "G. Wray Gill"},
        {"label": "PERSON", "pattern": "Gill", "id": "G. Wray Gill"},
    ],
    'rtownley' : [ #richard townley, nbc news reporter
        {"label": "PERSON", "pattern": "Richard Townley", "id": "Richard Townley"},
        {"label": "PERSON", "pattern": "Townley", "id": "Richard Townley"},
    ],
    'torres' : [ # miguel torres, inmate
        {"label": "PERSON", "pattern": "Miguel Torres", "id": "Miguel Torres"},
        {"label": "PERSON", "pattern": "Torres", "id": "Miguel Torres"},
    ],
    'aclemons' : [ # acquilla clemons, witness jd tippitt shooting
        {"label": "PERSON", "pattern": "Acquilla Clemons", "id": "Acquilla Clemons"},
        {"label": "PERSON", "pattern": "Clemons", "id": "Acquilla Clemons"},
    ],
    'jmpoe' : [ # j.m. poe, dallas pd
        {"label": "PERSON", "pattern": "J.M. Poe", "id": "J.M. Poe"},
        {"label": "PERSON", "pattern": "Poe", "id": "J.M. Poe"},
        {"label": "PERSON", "pattern": "Officer Poe", "id": "J.M. Poe"},
    ],
    'brading' : [ #eugene hale brading, incl aka jim braden
        {"label": "PERSON", "pattern": "Jim Braden", "id": "Eugen Hale Brading"},
        {"label": "PERSON", "pattern": "Braden", "id": "Eugen Hale Brading"},
        {"label": "PERSON", "pattern": "Eugene Hale Brading", "id": "Eugen Hale Brading"},
    ],
    'rmatthews' : [ # richard matthews, shaw defense attorney team, incl matthews
        {"label": "PERSON", "pattern": "Richard Matthews", "id": "Richard Matthews"},
        {"label": "PERSON", "pattern": "Matthews", "id": "Richard Matthews"},
    ],
    'gallinghouse' : [ # gerald gallinghouse, attorney
        {"label": "PERSON", "pattern": "Gerald Gallinghouse", "id": "Gerald Gallinghouse"},
        {"label": "PERSON", "pattern": "Gallinghouse", "id": "Gerald Gallinghouse"},
    ],
    'rankin' : [ #james lee rankin, attorney, counsel to warren comm
        {"label": "PERSON", "pattern": "J. Lee Rankin", "id": "James Lee Rankin"},
        {"label": "PERSON", "pattern": "Rankin", "id": "James Lee Rankin"},
    ],
    'specter' : [ #arlen specter, penn senator, counsel to warr comm
        {"label": "PERSON", "pattern": "Arlen Specter", "id": "Arlen Specter"},
        {"label": "PERSON", "pattern": "Specter", "id": "Arlen Specter"},
    ],
    'liebeler' : [ #wesley james liebeler, lawyer, warr comm staff
        {"label": "PERSON", "pattern": "Wesley Liebeler", "id": "Wesley Liebeler"},
        {"label": "PERSON", "pattern": "Liebeler", "id": "Wesley Liebeler"},
        {"label": "PERSON", "pattern": "Wesley J. Liebeler", "id": "Wesley Liebeler"},
    ],
    'redlich' : [ #norman redlich, lawyer, warrcomm staff, not incl norman
        {"label": "PERSON", "pattern": "Norman Redlich", "id": "Norman Redlich"},
        {"label": "PERSON", "pattern": "Redlich", "id": "Norman Redlich"},
        #{"label": "PERSON", "pattern": "Norman", "id": "Norman Redlich"},#in lane rush refers to harold norman
    ],
    'jball' : [ #joseph a. ball, lawyer, counsel warrcomm, incl "Ball"
        {"label": "PERSON", "pattern": "Joseph Ball", "id": "Joseph A. Ball"},
        {"label": "PERSON", "pattern": [{"TEXT": "Ball"}], "id": "Joseph A. Ball"},
    ],
    'rfrazier' : [ #robert frazier, fbi ballistics, not incl Frazier which primarily mean WBFrazier
        {"label": "PERSON", "pattern": "Robert A. Frazier", "id": "Robert Frazier"},
        {"label": "PERSON", "pattern": "FBI expert Frazier", "id": "Robert Frazier"},
    ],
    'willens' : [ #howard penney willens, lawyer, counsel warrcomm
        {"label": "PERSON", "pattern": "Howard P. Willens", "id": "Howard Penney Willens"},
        {"label": "PERSON", "pattern": "Howard Penney Willens", "id": "Howard Penney Willens"},
        {"label": "PERSON", "pattern": "Willens", "id": "Howard Penney Willens"},
        {"label": "PERSON", "pattern": "Howard Willens", "id": "Howard Penney Willens"},
    ],
    'brennan' : [ #howard brennan, writer, witness JFK shooting
        {"label": "PERSON", "pattern": "Howard L. Brennan", "id": "Howard Leslie Brennan"},
        {"label": "PERSON", "pattern": "Howard Brennan", "id": "Howard Leslie Brennan"},
        {"label": "PERSON", "pattern": "Brennan", "id": "Howard Leslie Brennan"},
    ], 
    'gford' : [ #gerald r ford, former us pres, warrcomm, incl "Ford"
        {"label": "PERSON", "pattern": "Gerald R. Ford", "id": "Gerald Ford"},
        {"label": "PERSON", "pattern": "Commissioner Ford", "id": "Gerald Ford"},
        {"label": "PERSON", "pattern": "Representative Ford", "id": "Gerald Ford"},
        #{"label": "PERSON", "pattern": [{"TEXT": "Ford"}], "id": "Gerald Ford"},
        {"label": "PERSON", "pattern": "Congressman Ford", "id": "Gerald Ford"},
    ],
    'goldberg' : [ #alfred goldberg, historian, coauthor/coeditor warrcomm report
        {"label": "PERSON", "pattern": "Alfred Goldberg", "id": "Alfred Goldberg"},
        {"label": "PERSON", "pattern": "Goldberg", "id": "Alfred Goldberg"},
    ],
    'cwalther' : [ #carolyn walther, freq referred to as mrs walther, witness jfk shooting
        {"label": "PERSON", "pattern": "Mrs Eric Walther", "id": "Carolyn Walther"},
        {"label": "PERSON", "pattern": "Mrs Walther", "id": "Carolyn Walther"},
    ],
    'mccloy' : [ #john j mccloy, lawyer, banker, warrcomm
        {"label": "PERSON", "pattern": "John J. McCloy", "id": "John J. McCloy"},
        {"label": "PERSON", "pattern": "McCloy", "id": "John J. McCloy"},
        {"label": "PERSON", "pattern": "Commissioner McCloy", "id": "John J. McCloy"},
    ],
    'rrussell' : [ #richard russell jr., warrcomm, incl russell
        {"label": "PERSON", "pattern": "Richard B. Russell", "id": "Richard Russell Jr."},
        {"label": "PERSON", "pattern": "Richard Russell", "id": "Richard Russell Jr."},
        {"label": "PERSON", "pattern": "Senator Russell", "id": "Richard Russell Jr."},
        {"label": "PERSON", "pattern": "Russell", "id": "Richard Russell Jr."},
    ],
    'rowland' : [ #arnold louis rowland, witness jfk shooting
        {"label": "PERSON", "pattern": "Arnold Rowland", "id": "Arnold Rowland"},
        {"label": "PERSON", "pattern": "Rowland", "id": "Arnold Rowland"},
    ],
    'ddryder' : [ #dial d ryder, dallas gunsmith
        {"label": "PERSON", "pattern": "Dial D. Ryder", "id": "Dial D. Ryder"},
        {"label": "PERSON", "pattern": "Ryder", "id": "Dial D. Ryder"},
    ],
    'hwade' : [ #henry wade, roe v wade, Dallas DA, tried ruby
        {"label": "PERSON", "pattern": "Henry Wade", "id": "Henry Wade"},
        {"label": "PERSON", "pattern": "Wade", "id": "Henry Wade"},
    ],
    'belin' : [ # david w belin, counsel warrcomm
        {"label": "PERSON", "pattern": "David W. Belin", "id": "David W. Belin"},
        {"label": "PERSON", "pattern": "Belin", "id": "David W. Belin"},
        {"label": "PERSON", "pattern": "David Belin", "id": "David W. Belin"},
    ],
    'griffin' : [ #burt w griffin, counsel warrcomm
        {"label": "PERSON", "pattern": "Burt W. Griffin", "id": "Burt Griffin"},
        {"label": "PERSON", "pattern": "Burt Griffin", "id": "Burt Griffin"},
        {"label": "PERSON", "pattern": "Griffin", "id": "Burt Griffin"},
    ],
    'hudkins' : [ #alonzo hudkins, houston newspaper report
        {"label": "PERSON", "pattern": "Alonzo Hudkins", "id": "Alonzo Hudkins"},
        {"label": "PERSON", "pattern": "Hudkins", "id": "Alonzo Hudkins"},
    ],
    'odio' : [ #sylvia odio, anticastro activist
        {"label": "PERSON", "pattern": "Sylvia Odio", "id": "Sylvia Odio"},
        {"label": "PERSON", "pattern": "Mrs Odio", "id": "Sylvia Odio"},
        {"label": "PERSON", "pattern": "Mrs Sylvia Odio", "id": "Sylvia Odio"},
        {"label": "PERSON", "pattern": "Odio", "id": "Sylvia Odio"},
    ],
    'eisenberg' : [ #melvin a eisenberg, lawyer, counsel warrcomm
        {"label": "PERSON", "pattern": "Melvin A. Eisenberg", "id": "Melvin A. Eisenberg"},
        {"label": "PERSON", "pattern": "Melvin Eisenberg", "id": "Melvin A. Eisenberg"},
        {"label": "PERSON", "pattern": "Eisenberg", "id": "Melvin A. Eisenberg"},
    ],
    'jcooper' : [ #john sherman cooper, kentucky politician, warrcomm, incl cooper
        {"label": "PERSON", "pattern": "John Sherman Cooper", "id": "John Sherman Cooper"},
        {"label": "PERSON", "pattern": "John Cooper", "id": "John Sherman Cooper"},
        {"label": "PERSON", "pattern": "Senator Cooper", "id": "John Sherman Cooper"},
        {"label": "PERSON", "pattern": "Cooper", "id": "John Sherman Cooper"},
    ],
    'tomlinson' : [ #darrell tomlinson, engineer, park hospital
        {"label": "PERSON", "pattern": "Darrell Tomlinson", "id": "Darrell Tomlinson"},
        {"label": "PERSON", "pattern": "Tomlinson", "id": "Darrell Tomlinson"},
    ],
    'ewalker' : [ #edwin walker, general, oswald attempted to assassinate, not incl walker
        {"label": "PERSON", "pattern": "Edwin Walker", "id": "Edwin Walker"},
        {"label": "PERSON", "pattern": "General Walker", "id": "Edwin Walker"},
    ],
    'slawson' : [ #w. david slawson, counsel warrcomm
        {"label": "PERSON", "pattern": "W. David Slawson", "id": "W. David Slawson"},
        {"label": "PERSON", "pattern": "Slawson", "id": "W. David Slawson"},
    ],
    'boggs' : [ #hale boggs, louisianna politician, warrcomm
        {"label": "PERSON", "pattern": "Hale Boggs", "id": "Hale Boggs"},
        {"label": "PERSON", "pattern": "Representative Boggs", "id": "Hale Boggs"},
        {"label": "PERSON", "pattern": "Boggs", "id": "Hale Boggs"},
    ],
    'fwhadams' : [ #francis w. h. adams, nycpd commissioner, counsel warrcomm, not incl adams
        {"label": "PERSON", "pattern": "Francis W. H. Adams", "id": "Francis W. H. Adams"},
        {"label": "PERSON", "pattern": "Francis Adams", "id": "Francis W. H. Adams"},
    ],
    'tdillard' : [ # tom dillard, dallas reporter
        {"label": "PERSON", "pattern": "Tom Dillard", "id": "Tom Dillard"},
        {"label": "PERSON", "pattern": "Dillard", "id": "Tom Dillard"},
    ],
    'olivier' : [ #alfred olivier, doctor, ballistics
        {"label": "PERSON", "pattern": "Alfred Olivier", "id": "Alfred Olivier"},
        {"label": "PERSON", "pattern": "Olivier", "id": "Alfred Olivier"},
        {"label": "PERSON", "pattern": "Doctor Olivier", "id": "Alfred Olivier"},
        {"label": "PERSON", "pattern": "Dr Olivier", "id": "Alfred Olivier"},
        {"label": "PERSON", "pattern": "Doctor Alfred Olivier", "id": "Alfred Olivier"},
    ],
        'artemov' : [ #professor artemov, russian professor
        {"label": "PERSON", "pattern": "Professor Artemov", "id": "Professor Artemov"},
        {"label": "PERSON", "pattern": "Artemov", "id": "Professor Artemov"},
    ],
    'kilgallen' : [ #dorothy kilgallen, journalist, incl dorothy
        {"label": "PERSON", "pattern": "Dorothy Kilgallen", "id": "Dorothy Kilgallen"},
        {"label": "PERSON", "pattern": "Kilgallen", "id": "Dorothy Kilgallen"},
    ],
    'vasiliev' : [ #leonid vasiliev, russian psychologist parapsychologist
        {"label": "PERSON", "pattern": "Leonid Vasiliev", "id": "Leonid Vasiliev"},
        {"label": "PERSON", "pattern": "L. L. Vasiliev", "id": "Leonid Vasiliev"},
        {"label": "PERSON", "pattern": "Professor Vasiliev", "id": "Leonid Vasiliev"},
        {"label": "PERSON", "pattern": "Vasiliev", "id": "Leonid Vasiliev"},
    ],
    'allhoff' : [ #fred allhoff, author
        {"label": "PERSON", "pattern": "Fred Allhoff", "id": "Fred Allhoff"},
        {"label": "PERSON", "pattern": "Allhoff", "id": "Fred Allhoff"},
        {"label": "PERSON", "pattern": "Mr Allhoff", "id": "Fred Allhoff"},
    ],
    'guttmacher' : [ #manfred guttmacher, american psychiatrist
        {"label": "PERSON", "pattern": "Dr Manfred S. Guttmacher", "id": "Manfred Guttmacher"},
        {"label": "PERSON", "pattern": "Manfred Guttmacher", "id": "Manfred Guttmacher"},
        {"label": "PERSON", "pattern": "Dr Guttmacher", "id": "Manfred Guttmacher"},
        {"label": "PERSON", "pattern": "Guttmacher", "id": "Manfred Guttmacher"},
    ],
    'belli' : [ #melvin belli, american lawyer, ruby's attorney
        {"label": "PERSON", "pattern": "Melvin Belli", "id": "Melvin Belli"},
        {"label": "PERSON", "pattern": "Belli", "id": "Melvin Belli"},
    ],
    'wortendyke' : [ #reynier wortendyke, new jersey judge
        {"label": "PERSON", "pattern": "Judge Wortendyke", "id": "Reynier Wortendyke"},
        {"label": "PERSON", "pattern": "Reynier Wortendyke", "id": "Reynier Wortendyke"},
    ],
    'hlhunt' : [#h.l. hunt, american oil tycoon
        {"label": "PERSON", "pattern": "Haroldson Lafayette Hunt", "id": "H. L. Hunt, Jr."},
        {"label": "PERSON", "pattern": "H.L. Hunt", "id": "H. L. Hunt, Jr."},
        {"label": "PERSON", "pattern": "Mr Hunt", "id": "H. L. Hunt, Jr."},
        {"label": "PERSON", "pattern": "Hunt", "id": "H. L. Hunt, Jr."},
    ],
    'curry' : [ #jesse curry, dallas pd chief
        {"label": "PERSON", "pattern": "J.P. Curry", "id": "Jesse Curry"},
        {"label": "PERSON", "pattern": "Curry", "id": "Jesse Curry"},
        {"label": "PERSON", "pattern": "Chief Curry", "id": "Jesse Curry"},
    ],
    'abelincoln' : [ #abraham lincoln, us pres, incl lincoln
        {"label": "PERSON", "pattern": "Abraham Lincoln", "id": "Abraham Lincoln"},
        {"label": "PERSON", "pattern": "President Lincoln", "id": "Abraham Lincoln"},
        {"label": "PERSON", "pattern": "Lincoln", "id": "Abraham Lincoln"},
    ],
    'mcnamara' : [ #robert mcnamara, us sec of defense
        {"label": "PERSON", "pattern": "Robert McNamara", "id": "Robert McNamara"},
        {"label": "PERSON", "pattern": "McNamara", "id": "Robert McNamara"},
    ],
    'jjwilliams' : [ #john j williams, delaware senator, not incl williams
        {"label": "PERSON", "pattern": "Senator Williams", "id": "John J. Williams"},
    ],
    'fdr' : [ #franklin delano roosevelt, us pres, not incl roosevelt
        {"label": "PERSON", "pattern": "Franklin D. Roosevelt", "id": "Franklin Delano Roosevelt"},
        {"label": "PERSON", "pattern": "Franklin Roosevelt", "id": "Franklin Delano Roosevelt"},
        {"label": "PERSON", "pattern": "Franklin Delano Roosevelt", "id": "Franklin Delano Roosevelt"},
        {"label": "PERSON", "pattern": "Franklin Roosevelt", "id": "Franklin Delano Roosevelt"},
    ],
    'troosevelt' : [ #theodore roosevelt, us pres, not incl roosevelt, incl theodore
        {"label": "PERSON", "pattern": "Theodore Roosevelt", "id": "Theodore Roosevelt"},
        {"label": "PERSON", "pattern": "Teddy Roosevelt", "id": "Theodore Roosevelt"},
        {"label": "PERSON", "pattern": "Theodore", "id": "Theodore Roosevelt"},
    ],
    'cohn' : [# roy cohn, american lawyer
        {"label": "PERSON", "pattern": "Roy Cohn", "id": "Roy Cohn"},
        {"label": "PERSON", "pattern": "Cohn", "id": "Roy Cohn"},
    ],
    'truman' : [ #harry s truman, us president, incl truman
        {"label": "PERSON", "pattern": "Harry Truman", "id": "Harry S. Truman"},
        {"label": "PERSON", "pattern": "Truman", "id": "Harry S. Truman"},
    ],
    'degaulle' : [ #charles de gaulle, french president
        {"label": "PERSON", "pattern": "Charles de Gaulle", "id": "Charles de Gaulle"},
        {"label": "PERSON", "pattern": "de Gaulle", "id": "Charles de Gaulle"},
        {"label": "PERSON", "pattern": "De Gaulle", "id": "Charles de Gaulle"},
    ],
    'odonnell' : [ #kenneth odonnell, advisor to jfk
        {"label": "PERSON", "pattern": "Kenneth O'Donnell", "id": "Kenneth O'Donnell"},
        {"label": "PERSON", "pattern": "Ken O'Donnell", "id": "Kenneth O'Donnell"},
        {"label": "PERSON", "pattern": "O'Donnell", "id": "Kenneth O'Donnell"},
    ],
    'lawson' : [ #winston lawson, us secret service agent
        {"label": "PERSON", "pattern": "Winston Lawson", "id": "Winston Lawson"},
        {"label": "PERSON", "pattern": "Winston G. Lawson", "id": "Winston Lawson"},
        {"label": "PERSON", "pattern": "Lawson", "id": "Winston Lawson"},
    ],
    'jfdulles' : [ #john foster dulles, us secretary of state
        {"label": "PERSON", "pattern": "John Foster Dulles", "id": "John Foster Dulles"},
    ],
    'khrushchev' : [ #nikita sergeyevich khrushchev, ussr leader
        {"label": "PERSON", "pattern": "Khrushchev", "id": "Nikita Sergeyevich Khrushchev"},
        {"label": "PERSON", "pattern": "Nikita Khrushchev", "id": "Nikita Sergeyevich Khrushchev"},
        {"label": "PERSON", "pattern": "Chairman Khrushchev", "id": "Nikita Sergeyevich Khrushchev"},
    ],
    'mccone' : [ #john a. mccone, director CIA
        {"label": "PERSON", "pattern": "John McCone", "id": "John A. McCone"},
        {"label": "PERSON", "pattern": "McCone", "id": "John A. McCone"},
    ],
    'nixon' : [ #richard nixon, us president, incl nixon
        {"label": "PERSON", "pattern": "Richard Nixon", "id": "Richard Nixon"},
        {"label": "PERSON", "pattern": "Nixon", "id": "Richard Nixon"},
    ],
    'mccarthy' : [ #joseph mccarthy, us politician
        {"label": "PERSON", "pattern": "Joseph McCarthy", "id": "Joseph McCarthy"},
        {"label": "PERSON", "pattern": "McCarthy", "id": "Joseph McCarthy"},
        {"label": "PERSON", "pattern": "Senator McCarthy", "id": "Joseph McCarthy"},
    ],
    'rfrost' : [ #robert frost, american poet, not incl frost
        {"label": "PERSON", "pattern": "Robert Frost", "id": "Robert Frost"},
    ],
    'tiberius' : [ #tiberius gracchus, roman politician
        {"label": "PERSON", "pattern": "Tiberius Gracchus", "id": "Tiberius Gracchus"},
        {"label": "PERSON", "pattern": "Tiberius", "id": "Tiberius Gracchus"},
    ],
    'mosaddegh' : [ #mohammed mosaddegh, iranian prime minister
        {"label": "PERSON", "pattern": "Mohammed Mosaddegh", "id": "Mohammed Mosaddegh"},
        {"label": "PERSON", "pattern": "Mosaddegh", "id": "Mohammed Mosaddegh"},
        {"label": "PERSON", "pattern": "Prime Minister Mosaddegh", "id": "Mohammed Mosaddegh"},
    ],
    'stauffenberg' : [ #claus von stauffenberg, german soldier
        {"label": "PERSON", "pattern": "Klaus Von Stauffenberg", "id": "Claus von Stauffenberg"},
        {"label": "PERSON", "pattern": "Von Stauffenberg", "id": "Claus von Stauffenberg"},
        {"label": "PERSON", "pattern": "Colonel Von Stauffenberg", "id": "Claus von Stauffenberg"},
    ],
    'humphrey' : [ #hubert humphrey, us vice president
        {"label": "PERSON", "pattern": "Hubert Humphrey", "id": "Hubert Humphrey"},
        {"label": "PERSON", "pattern": "Humphrey", "id": "Hubert Humphrey"},
        {"label": "PERSON", "pattern": "Hubert H. Humphrey", "id": "Hubert Humphrey"},
        {"label": "PERSON", "pattern": "Senator Humphrey", "id": "Hubert Humphrey"},
    ], 
    'rockefeller' : [ #john d. rockefeller, us businessman, incl rockefeller
        {"label": "PERSON", "pattern": "John D. Rockefeller", "id": "John D. Rockefeller"},
        {"label": "PERSON", "pattern": "Rockefeller", "id": "John D. Rockefeller"},
        {"label": "PERSON", "pattern": "John D. Rockefeller", "id": "John D. Rockefeller"},
    ],
    'goldwater' : [ #barry goldwater, us politician
        {"label": "PERSON", "pattern": "Barry Goldwater", "id": "Barry Goldwater"},
        {"label": "PERSON", "pattern": "Goldwater", "id": "Barry Goldwater"},
    ],
    #'bbaker' : [ #bobby baker, political advisor, incl baker # not referred to by lane
       # {"label": "PERSON", "pattern": "Bobby Baker", "id": "Bobby Baker"},
     #   {"label": "PERSON", "pattern": "Baker", "id": "Bobby Baker"},
   # ],
    'fulbright' : [ #j. william fulbright, us senator
        {"label": "PERSON", "pattern": "J. William Fulbright", "id": "J. William Fulbright"},
        {"label": "PERSON", "pattern": "Senator Fulbright", "id": "J. William Fulbright"},
        {"label": "PERSON", "pattern": "Fulbright", "id": "J. William Fulbright"},
    ],
    'blough' : [ #roger blough, us businessman, incl blough
        {"label": "PERSON", "pattern": "Roger Blough", "id": "Roger Blough"},
        {"label": "PERSON", "pattern": "Mr Blough", "id": "Roger Blough"},
        {"label": "PERSON", "pattern": "Blough", "id": "Roger Blough"},
        {"label": "PERSON", "pattern": "Roger W. Blough", "id": "Roger Blough"},
    ],
    'kellerman' : [ #roy kellerman, us secret service agent
        {"label": "PERSON", "pattern": "Roy Kellerman", "id": "Roy Kellerman"},
        {"label": "PERSON", "pattern": "Kellerman", "id": "Roy Kellerman"},
        {"label": "PERSON", "pattern": "Roy H.Kellerman", "id": "Roy Kellerman"},
    ],
    'hitler' : [ #adolf hitler, german dictator
        {"label": "PERSON", "pattern": "Adolf Hitler", "id": "Adolf Hitler"},
        {"label": "PERSON", "pattern": "Hitler", "id": "Adolf Hitler"},
    ],
    'astevenson' : [ #adlai stevenson II, american politician, incl stevenson
        {"label": "PERSON", "pattern": "Adlai Stevenson", "id": "Adlai Stevenson II"},
        {"label": "PERSON", "pattern": "Adlai E. Stevenson", "id": "Adlai Stevenson II"},
        {"label": "PERSON", "pattern": "Stevenson", "id": "Adlai Stevenson II"},
    ],
    'gwashington' : [ #george washington, us president, not incl washington
        {"label": "PERSON", "pattern": "George Washington", "id": "George Washington"},
    ],
    'ddillon' : [ #douglas dillon, american diplomat, not incl dillon
        {"label": "PERSON", "pattern": "Douglas Dillon", "id": "Douglas Dillon"},
    ],
    'ward' : [ #chester ward, admiral us navy
        {"label": "PERSON", "pattern": "Admiral Chester Ward", "id": "Chester Ward"},
        {"label": "PERSON", "pattern": "Admiral Ward", "id": "Chester Ward"},
    ],
    'boliver' : [ #simon bolivar, venezuelan revolutionary, incl bolivar
        {"label": "PERSON", "pattern": "Simon Bolivar", "id": "Simon Bolivar"},
        {"label": "PERSON", "pattern": "Bolivar", "id": "Simon Bolivar"},
    ],
    'friedman' : [ #milton friedman, american economist, incl friedman
        {"label": "PERSON", "pattern": "Milton Friedman", "id": "Milton Friedman"},
        {"label": "PERSON", "pattern": "Friedman", "id": "Milton Friedman"},
    ],
    'macarthur' : [ #douglas macarthur, us army general, incl macarthur
        {"label": "PERSON", "pattern": "Douglas MacArthur", "id": "Douglas MacArthur"},
        {"label": "PERSON", "pattern": "MacArthur", "id": "Douglas MacArthur"},
        {"label": "PERSON", "pattern": "General MacArthur", "id": "Douglas MacArthur"},
    ],
    'gehlen' : [ #reinhard gehlen, german intelligence officer
        {"label": "PERSON", "pattern": "Reinhard Gehlen", "id": "Reinhard Gehlen"},
        {"label": "PERSON", "pattern": "Gehlen", "id": "Reinhard Gehlen"},
    ],
    'buell' : [ #buell wesley frazier, coworker of oswald, incl Frazier
        {"label": "PERSON", "pattern": "Buell Wesley Frazier", "id": "Buell Wesley Frazier"},
        {"label": "PERSON", "pattern": "Frazier", "id": "Buell Wesley Frazier"},
    ],
    'whaley' : [ # cab driver who drove oswald after the murder
        {"label": "PERSON", "pattern": "William Whaley", "id": "William Whaley"},
        {"label": "PERSON", "pattern": "Whaley", "id": "William Whaley"},
        {"label": "PERSON", "pattern": "William Wayne Whaley", "id": "William Whaley"},
    ],
#    "marrion" : [ #marrion l. baker, police officer present at jfk shooting, not referred to by Lane
#        {"label": "PERSON", "pattern": "Marrion L. Baker", "id": "Marrion L. Baker"},
#        {"label": "PERSON", "pattern": "Baker", "id": "Marrion L. Baker"},
#    ],
    "patdean" : [ # Patrick Dean, police sgt in charge of securing area of oswalds exchange, incl Dean for this book
        {"label": "PERSON", "pattern": "Patrick T. Dean", "id": "Patrick T. Dean"},
        {"label": "PERSON", "pattern": "Dean", "id": "Patrick T. Dean"},
        {"label": "PERSON", "pattern": "Sargeant Dean", "id": "Patrick T. Dean"},
    ],
    "mrsconnally" : [ # Idanell brill connally, wife of gov john connally
        {"label": "PERSON", "pattern": "Mrs Connally", "id": "Idanell Brill Connally"},
        {"label": "PERSON", "pattern": "Nellie Connally", "id": "Idanell Brill Connally"},
    ],
    "bringuier" : [ #carlos bringuier, cuban exile opposed to castro, sort of feud with oswald
        {"label": "PERSON", "pattern": "Carlos Bringuier", "id": "Carlos Bringuier"},
        {"label": "PERSON", "pattern": "Bringuier", "id": "Carlos Bringuier"},
    ],
    "shaneyfelt" : [ #lyndal l. shaneyfelt, fbi agent
        {"label": "PERSON", "pattern": "Lyndal L Shaneyfelt", "id": "Lyndall L. Shaneyfelt"},
        {"label": "PERSON", "pattern": "Shaneyfelt", "id": "Lyndall L. Shaneyfelt"},
    ],
    "kelley" : [ #thomas j kelley, liaison to warren commission on behalf of the secret service
        {"label": "PERSON", "pattern": "Thomas J. Kelley", "id": "Thomas J. Kelley"},
        {"label": "PERSON", "pattern": "Inspector Kelley", "id": "Thomas J. Kelley"},
        {"label": "PERSON", "pattern": "Kelley", "id": "Thomas J. Kelley"},
    ],
    "drperry" : [#malcolm perry, doctor to attend to both jfk and oswald after they were shot
        {"label": "PERSON", "pattern": "Dr Malcom Perry", "id": "Malcolm Perry"},
        {"label": "PERSON", "pattern": "Doctor Malcom Perry", "id": "Malcolm Perry"},
        {"label": "PERSON", "pattern": "Malcom Perry", "id": "Malcolm Perry"},
        {"label": "PERSON", "pattern": "Dr Perry", "id": "Malcolm Perry"},
        {"label": "PERSON", "pattern": "Doctor Perry", "id": "Malcolm Perry"},
        {"label": "PERSON", "pattern": "Perry", "id": "Malcolm Perry"},
    ],
    "mrsreid" : [ #mrs r. a. reid, witness oswald leave book repository
        {"label": "PERSON", "pattern": "Mrs Robert A. Reid", "id": "Mrs. R. A. Reid"},
        {"label": "PERSON", "pattern": "Mrs Reid", "id": "Mrs. R. A. Reid"},
    ],
    "altgens" : [ #james w. altgens, ap photojournalist at jfk assassination
        {"label": "PERSON", "pattern": "James W. Altgens", "id": "James William Altgens"},
        {"label": "PERSON", "pattern": "Ike Altgens", "id": "James William Altgens"},
        {"label": "PERSON", "pattern": "Altgens", "id": "James William Altgens"},
    ],
    "bledsoe" : [ #mary e bledsoe, saw oswald on bus after assassination, incl bledsoe
        {"label": "PERSON", "pattern": "Mary E. Bledsoe", "id": "Mary E. Bledsoe"},
        {"label": "PERSON", "pattern": "Mrs Bledsoe", "id": "Mary E. Bledsoe"},
        {"label": "PERSON", "pattern": "Mary Bledsoe", "id": "Mary E. Bledsoe"},
        {"label": "PERSON", "pattern": "Bledsoe", "id": "Mary E. Bledsoe"},
    ],
    "dougherty" : [ #jack dougherty, book depository employee
        {"label": "PERSON", "pattern": "Jack Dougherty", "id": "Jack Dougherty"},
        {"label": "PERSON", "pattern": "Dougherty", "id": "Jack Dougherty"},
    ],
    "mcdonald" : [ #m n mcdonald, arrested oswald at theater
        {"label": "PERSON", "pattern": "M. N. McDonald", "id": "M. N. McDonald"},
        {"label": "PERSON", "pattern": "McDonald", "id": "M. N. McDonald"},
    ],
    "gregory" : [ #dr charles gregory, treated gov connallys bullet wound
        {"label": "PERSON", "pattern": "Charles F. Gregory", "id": "Charles Francis Gregory"},
        {"label": "PERSON", "pattern": "Dr Gregory", "id": "Charles Francis Gregory"},
        {"label": "PERSON", "pattern": "Doctor Gregory", "id": "Charles Francis Gregory"},
    ],
    "stovall" : [ # r s stovall, dallas detective
        {"label": "PERSON", "pattern": "R. S. Stovall", "id": "R. S. Stovall"},
        {"label": "PERSON", "pattern": "Stovall", "id": "R. S. Stovall"},
    ],
    "mcwatters" : [ #cecil j mcwatters, bus driver
        {"label": "PERSON", "pattern": "Cecil J. McWatters", "id": "Cecil J. McWatters"},
        {"label": "PERSON", "pattern": "McWatters", "id": "Cecil J. McWatters"},
    ],
    "holmes" : [# harry d holmes, postal inspector, incl holmes
        {"label": "PERSON", "pattern": "Harry D. Holmes", "id": "Harry D. Holmes"},
        {"label": "PERSON", "pattern": "Holmes", "id": "Harry D. Holmes"},
    ],
    "robshaw" : [ #robert roeder shaw, tended to connallys wound, not incl shaw
        {"label": "PERSON", "pattern": "Robert Roeder Shaw", "id": "Robert Roeder Shaw"},
        {"label": "PERSON", "pattern": "Dr Shaw", "id": "Robert Roeder Shaw"},
        {"label": "PERSON", "pattern": "Doctor Shaw", "id": "Robert Roeder Shaw"},
    ],
    "bookhout" : [#james w bookhout, fbi agent
        {"label": "PERSON", "pattern": "Agent Bookhout", "id": "James W. Bookhout"},
        {"label": "PERSON", "pattern": "Bookhout", "id": "James W. Bookhout"},
    ],
    "randle" : [# linnie mae randle, sister to wesley frazier
        {"label": "PERSON", "pattern": "Linnie Mae Randle", "id": "Linnie Mae Randle"},
        {"label": "PERSON", "pattern": "Mrs Randle", "id": "Linnie Mae Randle"},
    ],
    "willis" : [ #phil willis, witness to jfk shooting, incl willis
        {"label": "PERSON", "pattern": "Phil Willis", "id": "Phil Willis"},
        {"label": "PERSON", "pattern": "Willis", "id": "Phil Willis"},
    ],
    "earlene" : [ #earlene roberts, housekeeper at building oswald lived, incl mrs. roberts, incl roberts
        {"label": "PERSON", "pattern": "Earlene Roberts", "id": "Earlene Roberts"},
        {"label": "PERSON", "pattern": "Mrs Roberts", "id": "Earlene Roberts"},
        {"label": "PERSON", "pattern": "Roberts", "id": "Earlene Roberts"},
    ],
    "carroll" : [ #bob carroll, officer involved with arrest of oswald, incl carroll
        {"label": "PERSON", "pattern": "Bob Carroll", "id": "Bob K. Carroll"},
        {"label": "PERSON", "pattern": "Carroll", "id": "Bob K. Carroll"},
    ],
    "snyder" : [ #richard e snyder, foreign service officer, handled oswalds defection, incl snyder
        {"label": "PERSON", "pattern": "Richard E. Snyder", "id": "Richard E. Snyder"},
        {"label": "PERSON", "pattern": "E. Snyder", "id": "Richard E. Snyder"},
        {"label": "PERSON", "pattern": "Snyder", "id": "Richard E. Snyder"},
    ],
    "decker" : [ #bill decker, dallas sheriff, part of jfk motorcade
        {"label": "PERSON", "pattern": "Sheriff Bill Decker", "id": "Bill Decker"},
        {"label": "PERSON", "pattern": "Bill Decker", "id": "Bill Decker"},
        {"label": "PERSON", "pattern": "Sheriff Decker", "id": "Bill Decker"},
        {"label": "PERSON", "pattern": "Decker", "id": "Bill Decker"},
    ],
    "evy" : [#evaristo rodriguez, bartender that served oswald meeting w a cuban, incl rodriguez
        {"label": "PERSON", "pattern": "Evaristo Rodriguez", "id": "Evaristo Rodriguez"},
        {"label": "PERSON", "pattern": "Rodriguez", "id": "Evaristo Rodriguez"},
    ],
    "sorrels" : [#forrest vernon sorrels, secret service agent based in dallas
        {"label": "PERSON", "pattern": "Forrest V. Sorrels", "id": "Forrest Vernon Sorrels"},
        {"label": "PERSON", "pattern": "Sorrels", "id": "Forrest Vernon Sorrels"},
        {"label": "PERSON", "pattern": "Agent Sorrels", "id": "Forrest Vernon Sorrels"},
        {"label": "PERSON", "pattern": "Forrest Sorrels", "id": "Forrest Vernon Sorrels"},
    ],
    "joesten" : [#joachim joesten, independent investigator and author
        {"label": "PERSON", "pattern": "Joachim Joesten", "id": "Joachim Joesten"},
        {"label": "PERSON", "pattern": "Joesten", "id": "Joachim Joesten"},
    ],
    "gfrose" : [#g.f. rose, detective dallas pd searched oswald home after shooting, incl Rise
        {"label": "PERSON", "pattern": "G. F. Rose", "id": "G. F. Rose"},
        {"label": "PERSON", "pattern": "Rose", "id": "G. F. Rose"},
    ],
    "vaughn" : [#roy e. vaughn, officer dallas pd
        {"label": "PERSON", "pattern": "R. E. Vaughn", "id": "Roy E. Vaughn"},
        {"label": "PERSON", "pattern": "Vaughn", "id": "Roy E. Vaughn"},
        {"label": "PERSON", "pattern": "Officer Vaughn", "id": "Roy E. Vaughn"},
    ],
    "weitzman" : [#seymour weitzman, dallas pd officer, one of two to find oswald rifle
        {"label": "PERSON", "pattern": "Seymour Weitzman", "id": "Seymour Weitzman"},
        {"label": "PERSON", "pattern": "Constable Weitzman", "id": "Seymour Weitzman"},
        {"label": "PERSON", "pattern": "Weitzman", "id": "Seymour Weitzman"},
    ],
    "nelson" : [#dorris mae nelson, nurse at parkland, ER supervisor
        {"label": "PERSON", "pattern": "Mrs Nelson", "id": "Dorris Mae Nelson"},
        {"label": "PERSON", "pattern": "Nurse Nelson", "id": "Dorris Mae Nelson"},
    ],
    "talbert" : [#Cecil e. talbert, capt dallas pd, coordinated police for oswald transfer
        {"label": "PERSON", "pattern": "Cecil E. Talbert", "id": "Cecil E. Talbert"},
        {"label": "PERSON", "pattern": "Captain Talbert", "id": "Cecil E. Talbert"},
        {"label": "PERSON", "pattern": "Talbert", "id": "Cecil E. Talbert"},
    ],
    "daniels" : [#napoleon j. daniels, witness ruby go into basement, incl daniels
        {"label": "PERSON", "pattern": "J. Daniels", "id": "Napoleon J. Daniels"},
        {"label": "PERSON", "pattern": "Daniels", "id": "Napoleon J. Daniels"},
        {"label": "PERSON", "pattern": "N. J. Daniels", "id": "Napoleon J. Daniels"},
    ],
    "benavides" : [#domingo benavides, witness tippit shooting
        {"label": "PERSON", "pattern": "Domingo Benavides", "id": "Domingo Benavides"},
        {"label": "PERSON", "pattern": "Benavides", "id": "Domingo Benavides"},
    ],
    "chaney" : [#james m. chaney, dallas pd, officer near jfk car at shooting
        {"label": "PERSON", "pattern": "Jim Chaney", "id": "James M. Chaney"},
        {"label": "PERSON", "pattern": "Chaney", "id": "James M. Chaney"},
    ],
    "craig" : [#walter early craig, us district judge arizona, not incl Craig
        {"label": "PERSON", "pattern": "Walter Craig", "id": "Walter Early Craig"},
        {"label": "PERSON", "pattern": "Mr Craig", "id": "Walter Early Craig"},
        {"label": "PERSON", "pattern": "Walter E. Craig", "id": "Walter Early Craig"},
    ],
    'mccoy' : [#c.c. mccoy, deputy dallas pd, received phone call threat on oswald before transfer, incl mccoy
        {"label": "PERSON", "pattern": "C. C. McCoy", "id": "C. C. McCoy"},
        {"label": "PERSON", "pattern": "McCoy", "id": "C. C. McCoy"},
    ],
    'boswell' : [#j. thornton boswell, physician, autopsy of jfk 
        {"label": "PERSON", "pattern": "J. Thornton Boswell", "id": "J. Thornton Boswell"},
        {"label": "PERSON", "pattern": "Dr Boswell", "id": "J. Thornton Boswell"},
        {"label": "PERSON", "pattern": "Boswell", "id": "J. Thornton Boswell"},
    ],
    'nprich' : [#nancy perrin rich, employed by Ruby as bartender, incl Rich
        {"label": "PERSON", "pattern": "Nancy Perrin Rich", "id": "Nancy Perrin Rich"},
        {"label": "PERSON", "pattern": "Mrs Rich", "id": "Nancy Perrin Rich"},
        {"label": "PERSON", "pattern": "Rich", "id": "Nancy Perrin Rich"},
    ],
    'brwilliams' : [#bonnie ray williams, employee at book depository, incl Williams
        {"label": "PERSON", "pattern": "Bonnie Ray Williams", "id": "Bonnie Ray Williams"},
        {"label": "PERSON", "pattern": "Williams", "id": "Bonnie Ray Williams"},
    ],
    'wtice' : [#wilma tice, claimed to see Ruby at Parkland when JFK went there, incl tice
        {"label": "PERSON", "pattern": "Wilma Tice", "id": "Wilma Tice"},
        {"label": "PERSON", "pattern": "Mrs Tice", "id": "Wilma Tice"},
        {"label": "PERSON", "pattern": "Tice", "id": "Wilma Tice"},
    ],
    'hnorman' : [#harold norman, employee at book depository, incl norman
        {"label": "PERSON", "pattern": "Harold Norman", "id": "Harold Norman"},
        {"label": "PERSON", "pattern": "Norman", "id": "Harold Norman"},
    ],
    'hidell' : [#alek james hidell, aka of oswald
        {"label": "PERSON", "pattern": "Alek James Hidell", "id": "Alek James Hidell"},
        {"label": "PERSON", "pattern": "A. J. Hidell", "id": "Alek James Hidell"},
        {"label": "PERSON", "pattern": "Hidell", "id": "Alek James Hidell"},
        {"label": "PERSON", "pattern": "Aleck Hidell", "id": "Alek James Hidell"},
    ],
    'mckeown' : [#robert mckeown, american gun smuggler to castro
        {"label": "PERSON", "pattern": "Robert R. McKeown", "id": "Robert McKeown"},
        {"label": "PERSON", "pattern": "McKeown", "id": "Robert McKeown"},
    ],
    'jarman' : [#james jarman jr, employee book depository
        {"label": "PERSON", "pattern": "James Jarman, Jr", "id": "James Jarman, Jr."},
        {"label": "PERSON", "pattern": "Jarman", "id": "James Jarman, Jr."},
    ],
    'latona' : [#sebastian francis latona, FBI fingerprint examiner
        {"label": "PERSON", "pattern": "Sebastian Francis Latona", "id": "Sebastian Francis Latona"},
        {"label": "PERSON", "pattern": "Latona", "id": "Sebastian Francis Latona"},
    ],
    'scoggins' : [#william scoggins, cab driver, tippit witness
        {"label": "PERSON", "pattern": "William Scoggins", "id": "William Scoggins"},
        {"label": "PERSON", "pattern": "Scoggins", "id": "William Scoggins"},
        {"label": "PERSON", "pattern": "William W. Scoggins", "id": "William Scoggins"},
    ],
    'kantor' : [#seth kantor, journalist, jfk witness
        {"label": "PERSON", "pattern": "Seth Kantor", "id": "Seth Kantor"},
        {"label": "PERSON", "pattern": "Kantor", "id": "Seth Kantor"},
    ],
    'wreynolds' : [#warren reynolds, witness to tippit, incl reynolds
        {"label": "PERSON", "pattern": "Warren Reynolds", "id": "Warren Reynolds"},
        {"label": "PERSON", "pattern": "Reynolds", "id": "Warren Reynolds"},
    ],
    'ewhitworth' :[#edith whitworth, furniture store operator
        {"label": "PERSON", "pattern": "Edith Whitworth", "id": "Edith Whitworth"},
        {"label": "PERSON", "pattern": "Mrs Whitworth", "id": "Edith Whitworth"},
    ],
    'bogard' : [#albert guy bogard, car salesman
        {"label": "PERSON", "pattern": "Albert G. Bogard", "id": "Albert Bogard"},
        {"label": "PERSON", "pattern": "Bogard", "id": "Albert Bogard"},
    ],
    'lovelady' : [#billy nolan lovelady, book depository employee jfk witness
        {"label": "PERSON", "pattern": "Billy Nolan Lovelady", "id": "Billy Nolan Lovelady"},
        {"label": "PERSON", "pattern": "Lovelady", "id": "Billy Nolan Lovelady"},
    ],
    'osborne' : [#albert osborne, allegedly rode bus with oswald to mex city, has alias john howard bowen not incl
        {"label": "PERSON", "pattern": "Albert Osborne", "id": "Albert Osborne"},
        {"label": "PERSON", "pattern": "Osborne", "id": "Albert Osborne"},
    ],
    'weissman' : [#bernard weissman, allegedly met with tippit and ruby in some conspiracy
        {"label": "PERSON", "pattern": "Bernard Weissman", "id": "Bernard Weissman"},
        {"label": "PERSON", "pattern": "Weissman", "id": "Bernard Weissman"},
        {"label": "PERSON", "pattern": "Mr Weissman", "id": "Bernard Weissman"},
    ],
    'marguerite' : [# marguerite oswald, mother of lee harvey
        {"label": "PERSON", "pattern": "Marguerite Oswald", "id": "Marguerite Oswald"}, 
        {"label": "PERSON", "pattern": "Oswald", "id": "Marguerite Oswald"},
    ],
    'simmons' : [ #james l simmons, jfk shooting witness, incl Simmons
        {"label": "PERSON", "pattern": "James L. Simmons", "id": "James L. Simmons"},
        {"label": "PERSON", "pattern": "Simmons", "id": "James L. Simmons"},
    ],
    'shanklin' : [# james gordon shanklin, FBI agent
        {"label": "PERSON", "pattern": "Gordon Shanklin", "id": "James Gordon Shanklin"},
        {"label": "PERSON", "pattern": "Agent Shanklin", "id": "James Gordon Shanklin"},
        {"label": "PERSON", "pattern": "Shanklin", "id": "James Gordon Shanklin"},
    ],
    'crafard' : [#larry crafard, employed by and friend of ruby
        {"label": "PERSON", "pattern": "Larry Crafard", "id": "Larry Crafard"},
        {"label": "PERSON", "pattern": "Crafard", "id": "Larry Crafard"},
    ],
    'bowers' : [#lee bowers, jfk shooting witness
        {"label": "PERSON", "pattern": "Lee Bowers", "id": "Lee Bowers"},
        {"label": "PERSON", "pattern": "Bowers", "id": "Lee Bowers"},
    ],
    'holland' : [# s. m. holland, jfk shooting witness, incl holland
        {"label": "PERSON", "pattern": "S. M. Holland", "id": "S. M. Holland"},
        {"label": "PERSON", "pattern": "Holland", "id": "S. M. Holland"},
    ],
    'ljlewis' : [#l.j. lewis, owned car lot, saw man fleeing after gunshots
        {"label": "PERSON", "pattern": "L. J. Lewis", "id": "L. J. Lewis"},
        {"label": "PERSON", "pattern": "Lewis", "id": "L. J. Lewis"},
    ],
    'ghunter' : [#gertrude hunter, friend of mrs whitworth, not incl hunter
        {"label": "PERSON", "pattern": "Gertrude Hunter", "id": "Gertrude Hunter"},
        {"label": "PERSON", "pattern": "Mrs Hunter", "id": "Gertrude Hunter"},
    ],
    'odum' : [#bardwell odum, FBI agent, incl hart odum as misidentified by marguerite oswald
        {"label": "PERSON", "pattern": "Bardwell D. Odum", "id": "Bardwell D. Odum"},
        {"label": "PERSON", "pattern": "Odum", "id": "Bardwell D. Odum"},
        {"label": "PERSON", "pattern": "Hart Odum", "id": "Bardwell D. Odum"},
    ],
    'mamoorman' : [#mary ann moorman, photographed jfk <second after being shot
        {"label": "PERSON", "pattern": "Mary Ann Moorman", "id": "Mary Ann Moorman"},
        {"label": "PERSON", "pattern": "Mrs Moorman", "id": "Mary Ann Moorman"},
    ],
    'leavelle' : [#james r leavelle, dallas pd detective escorting oswald when he was shot by ruby
        {"label": "PERSON", "pattern": "James R. Leavelle", "id": "James R. Leavelle"},
        {"label": "PERSON", "pattern": "Leavelle", "id": "James R. Leavelle"},
        {"label": "PERSON", "pattern": "Detective Leavelle", "id": "James R. Leavelle"},
    ],
    'vadams' : [#victoria adams, jfk shooting witness, not incl adams
        {"label": "PERSON", "pattern": "Victoria Elizabeth Adams", "id": "Victoria Elizabeth Adams"},
        {"label": "PERSON", "pattern": "Miss Adams", "id": "Victoria Elizabeth Adams"},
    ],
    'boone' : [#eugene boone, dallas pd, found oswalds rifle
        {"label": "PERSON", "pattern": "Eugene Boone", "id": "Eugene Boone"},
        {"label": "PERSON", "pattern": "Officer Boone", "id": "Eugene Boone"},
        {"label": "PERSON", "pattern": "Boone", "id": "Eugene Boone"},
    ],
    'cunningham' : [#cortlandt cunningham, fbi agent, ballistics, incl cunningham
        {"label": "PERSON", "pattern": "Cortlandt Cunningham", "id": "Cortlandt Cunningham"},
        {"label": "PERSON", "pattern": "Cunningham", "id": "Cortlandt Cunningham"},
    ],
    'evagrant' : [#eva grant, sister of jack ruby, nightclub manager, not incl grant
        {"label": "PERSON", "pattern": "Eva Grant", "id": "Eva Rubenstein Grant"},
        {"label": "PERSON", "pattern": "Mrs Grant", "id": "Eva Rubenstein Grant"},
    ],
    'lmrandle' : [#linnie mae randle, sister to wesley frazier, incl randle
        {"label": "PERSON", "pattern": "Linnie Mae Randle", "id": "Linnie Mae Randle"},
        {"label": "PERSON", "pattern": "Mrs Randle", "id": "Linnie Mae Randle"},
        {"label": "PERSON", "pattern": "Randle", "id": "Linnie Mae Randle"},
    ],
    'wseymour' : [#william seymour, cia agent, interpen, not incl seymour
        {"label": "PERSON", "pattern": "William Seymour", "id": "William Seymour"},
    ],
    'jhsawyer' : [#j herbert sawyer, dallas pd, incl sawyer
        {"label": "PERSON", "pattern": "J. Herbert Sawyer", "id": "J. Herbert Sawyer"},
        {"label": "PERSON", "pattern": "Sawyer", "id": "J. Herbert Sawyer"},
        {"label": "PERSON", "pattern": "Inspector Sawyer", "id": "J. Herbert Sawyer"},
    ],
    'eanderson' : [#eugene anderson, marine, testified on oswalds marksmanship, not incl anderson
        {"label": "PERSON", "pattern": "Eugene D. Anderson", "id": "Eugene D. Anderson"},
        {"label": "PERSON", "pattern": "Major Anderson", "id": "Eugene D. Anderson"},
    ],
    'lcgraves' : [#lc graves, dallas pd, took gun from ruby, incl graves
        {"label": "PERSON", "pattern": "L. C. Graves", "id": "L. C. Graves"},
        {"label": "PERSON", "pattern": "Graves", "id": "L. C. Graves"},
    ],
    'fferraro' : [#frank ferraro, worked for ruby, assaulted by ruby, incl ferraro
        {"label": "PERSON", "pattern": "Frank Ferraro", "id": "Frank Ferraro"},
        {"label": "PERSON", "pattern": "Ferraro", "id": "Frank Ferraro"},
    ],
    'lhoward' : [#lawrence howard, cia agent, interpen, not incl howard
        {"label": "PERSON", "pattern": "Lawrence Howard", "id": "Lawrence Howard"},
    ],
    'gslack' : [#garland g slack, pssibly witnessed oswald at a gun range, incl slack
        {"label": "PERSON", "pattern": "Garland G. Slack", "id": "Garland G. Slack"},
        {"label": "PERSON", "pattern": "Slack", "id": "Garland G. Slack"},
    ],
    'betzner' : [#hugh betzner, photographer, jfk witness
        {"label": "PERSON", "pattern": "Hugh Betzner", "id": "Hugh Betzner, Jr."},
        {"label": "PERSON", "pattern": "Betzner", "id": "Hugh Betzner, Jr."},
    ],
    #'ocampbell' : [#ochus v campbell, vp of depository building, witness jfk, incl campbell
    #    {"label": "PERSON", "pattern": "Ochus V. Campbell", "id": "Ochus V. Campbell"},
    #    {"label": "PERSON", "pattern": "Campbell", "id": "Ochus V. Campbell"},
    #],
    'mcwatters' : [#cj mcwatters, bus driver day of jfk shooting
        {"label": "PERSON", "pattern": "C. J. McWatters", "id": "Cecil J. McWatters"},
        {"label": "PERSON", "pattern": "McWatters", "id": "Cecil J. McWatters"},
    ],
    'kclark' : [#kemp clark, parkland doctor who declared jfk dead, not incl clark
        {"label": "PERSON", "pattern": "Kemp Clark", "id": "Kemp Clark"},
        {"label": "PERSON", "pattern": "Dr Clark", "id": "Kemp Clark"},
    ],
    'wshelley' : [#william shelley, employee depository building, incl shelley
        {"label": "PERSON", "pattern": "William H. Shelley", "id": "William H. Shelley"},
        {"label": "PERSON", "pattern": "William Shelley", "id": "William H. Shelley"},
        {"label": "PERSON", "pattern": "Shelley", "id": "William H. Shelley"},
    ],
    'nicol' : [#joseph d nicol, ballistics expert, testified to warren commission
        {"label": "PERSON", "pattern": "Joseph D. Nicol", "id": "Joseph D. Nicol"},
        {"label": "PERSON", "pattern": "Nicol", "id": "Joseph D. Nicol"},
    ],
    'rpierce' : [#rio pierce, dallas pd, present at oswald transfer, incl pierce
        {"label": "PERSON", "pattern": "Lieutenant Pierce", "id": "Rio S. Pierce"},
        {"label": "PERSON", "pattern": "Pierce", "id": "Rio S. Pierce"},
    ],
    'lschmidt' : [#larrie schmidt, right wing activist, incl schmidt
        {"label": "PERSON", "pattern": "Larrie Schmidt", "id": "Larrie Schmidt"},
        {"label": "PERSON", "pattern": "Schmidt", "id": "Larrie Schmidt"},
    ],
    'greener' : [#charles w greener, worked at rifle shop
        {"label": "PERSON", "pattern": "Charles W. Greener", "id": "Charles W. Greener"},
        {"label": "PERSON", "pattern": "Greener", "id": "Charles W. Greener"},
    ],
    'ekthompson' : [#edward k thompson, editor, life mag, founder smithsonian mag, incl thompson
        {"label": "PERSON", "pattern": "Edward K. Thompson", "id": "Edward K. Thompson"},
        {"label": "PERSON", "pattern": "Thompson", "id": "Edward K. Thompson"},
    ],
    'jhill' : [# jean hill, witness jfk shooting, lady in red, not incl hill
        {"label": "PERSON", "pattern": "Jean Hill", "id": "Jean Hill"},
        {"label": "PERSON", "pattern": "Mrs Hill", "id": "Jean Hill"},
    ]
}

In [ ]:
for p in ruler_patterns:
    ruler.add_patterns(ruler_patterns[p])

In [ ]:
# bring in the text and create doc container
with open ("Lane_Rush_body.txt", "r") as f:
    text = f.read()
doc = nlp(text)

In [ ]:
# print the entities of the text with label and id #(if one has been assigned)#
for ent in doc.ents:
    print (ent.text, ent.label_, ent.id_)

In [ ]:
# this sorts entities by their count so you can see the most common entities recognized
entity_counts = Counter([(ent.text, ent.label_) for ent in doc.ents])

print("Entity Counts (sorted by count):")
sorted_counts = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)
for (entity, label), count in sorted_counts:
    print(f"{entity} ({label}): {count}")

In [ ]:
# this is the same print and sort butyou can now see if an id has been assigned
# use this output to identify entities that need to be resolved to IDs and edit Entity Ruler accordingly
entity_counts = Counter([(ent.text, ent.id_) for ent in doc.ents])

print("Entity Counts (sorted by count):")
sorted_counts = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)
for (entity, label), count in sorted_counts:
    print(f"{entity} ({label}): {count}")

In [ ]:
# this section will show counts of entities resolved to an ID
# Count the occurrences of each entity id
entity_counter = Counter()
for ent in doc.ents:
    entity_counter[ent.id_] += 1

# Sort entity labels by frequency
sorted_labels = sorted(entity_counter.items(), key=lambda x: x[1], reverse=True)

# Print the sorted list of entity labels
for label, count in sorted_labels:
    print(f"Entity ID: {label}, Mentions: {count}")

In [ ]:
# based on this project https://github.com/thu-vu92/the_witcher_network/blob/main/extracting_relationships.ipynb 
# get named entity list per sentence
sent_ent_df = []

# loop through sentences, store named entity list for each sentence
for sent in doc.sents:
    entity_list = [ent.id_ for ent in sent.ents]
    sent_ent_df.append({"Sentence": sent, "Entities": entity_list})

sent_ent_df = pd.DataFrame(sent_ent_df)

# print the sent_ent_df, which takes a while to process
sent_ent_df

In [ ]:
## this creates a rolling window of 5 sentences
## update df name after figuring out the filtering mentioned above

window_size = 5
relationships = []
tally = {}
tally_df = pd.DataFrame(columns=['A', 'B', 'Count'])

for i in range(sent_ent_df.index[-1]):
    end_i = min(i+5, sent_ent_df.index[-1])
    char_list = sum((sent_ent_df.loc[i: end_i].Entities), [])
    
    # Remove duplicated characters that are next to each other
    char_unique = [char_list[i] for i in range(len(char_list)) 
                   if (i==0) or char_list[i] != char_list[i-1]]

    if len(char_unique) > 1:
        for idx, source in enumerate(char_unique[:-1]):
            target = char_unique[idx + 1]
            # print(i, idx, source, target)
            if source != '' and target != '':
                relationships.append({"source": source, "target": target})
                a,b=sorted((source,target))
                # Increment this pairing in the tally, or add it if it's not there
                if a in tally:
                    tally[a][b] = tally[a][b]+1 if b in tally[a] else 1
                else:
                    tally[a] = {b:1}
            
# Convert tally to a DF
for col1,count in tally.items():
    for col2 in count:
        tally_df.loc[len(tally_df.index)] = [col1, col2, count[col2]]

relationship_df = pd.DataFrame(relationships)

pd.set_option('display.max_rows', None)
# relationship_df
tally_df

In [ ]:
# Sort the cases with a->b and b->a
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df

In [ ]:
## add weights to the relationships by sum of occurrences

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()
relationship_df.head(50)

In [ ]:
relationship_df.to_csv('lane_rush_edges.csv', index=False)